# **Script para descargar los informes post partido de MediaCoach**

## **Pasos**:

1 - Conectar a Google Drive

2 - Obtener credenciales

3 - Definición de funciones

4 - Interfaz de usuario y ejecución del código

## **Importante**:

* La estructura de las carpetas donde se guardan los archivos debe ser la siguiente: /content/drive/MyDrive/VCF_Mediacoach_Data/Temporada_yy_yy/Competicion (Ej: /content/drive/MyDrive/VCF_Mediacoach_Data/Temporada_24_25/La_Liga).

* Dentro de la carpeta de cada competición, para cada temporada, tiene que haber un archivo csv con los ids de los partidos ya procedados. La estrcutura de las carpetas y nombre de este archivo debe ser el siguiente:
/content/drive/MyDrive/VCF_Mediacoach_Data/Temporada_yy_yy/ids_procesados_competicion.csv (Ej: /content/drive/MyDrive/VCF_Mediacoach_Data/Temporada_23_24/ids_procesados_La_Liga_2.csv)

# **2 - Obtener Credenciales**

In [10]:
import requests
import subprocess
import json
import zipfile
import csv
import os
import io

# Establece los parámetros necesarios para la solicitud
client_id = '58191b89-cee4-11ed-a09d-ee50c5eb4bb5'
scope = 'b2bapiclub-api'
grant_type = 'password'
username = 'b2bvillarealcf@mediacoach.es'
password = 'r728-FHj3RE!'

# URL a la que se hará la solicitud de token
token_url = 'https://id.mediacoach.es/connect/token'

# Datos que se enviarán en la solicitud
data = {
    'client_id': client_id,
    'scope': scope,
    'grant_type': grant_type,
    'username': username,
    'password': password
}

# Encabezados para la solicitud
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Realiza la solicitud POST
response = requests.post(token_url, data=data, headers=headers)

# Verifica si la solicitud fue exitosa
if response.status_code == 200:
    # Convierte la respuesta en JSON y obtiene el access_token
    token_response = response.json()
    AccessToken = token_response.get('access_token', '')
    expires_in = token_response.get('expires_in', '')
    token_type = token_response.get('token_type', '')
    scope = token_response.get('scope', '')

    # Imprime el access_token y otros detalles
    print(f'Access Token: {AccessToken}')
    print(f'Expires In: {expires_in}')
    print(f'Token Type: {token_type}')
    print(f'Scope: {scope}')
else:
    # Imprime el mensaje de error si la solicitud no fue exitosa
    print(f'Error: {response.status_code}')
    print(response.text)

#Subscripción del villarreal
SubscriptionKey = '729f9154234d4ff3bb0a692c6a0510c4'
#Url base de la api de media coach
api_url_base = "https://club-api.mediacoach.es"


# Función para hacer consultar curl
credenciales = f"--header 'Ocp-Apim-Subscription-Key: {SubscriptionKey}' --header 'Authorization: Bearer {AccessToken}'"

def ejecutar_curl_comando(comando):
    process = subprocess.Popen(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print("Error en el comando curl:")
        print(stderr.decode())
        return None
    return json.loads(stdout)


Access Token: 294ACB3F3F472ABE277C8F5DD9CFF70B4185AC6A31F6911483BA9AA8F2FB7AF8
Expires In: 10800
Token Type: Bearer
Scope: b2bapiclub-api


# **3- Definición de Funciones**

In [11]:
#Función que guarda los match_id tras leetlos del archivo ids_procesados.csv
def guardar_id_en_csv(id, temporada="Temporada_24_25", competicion="La_Liga", archivo_ids=""):
    nombre_archivo=f'./VCF_Mediacoach_Data/{temporada}/{competicion}/{archivo_ids}'
    with open(nombre_archivo, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([id])


#Función que devuelve los match_id tras leerlos del archivo ids_procesados.csv
def leer_ids_csv(temporada="Temporada_24_25", competicion="La_Liga", archivo_ids="ids_procesados_La_Liga.csv"):
    nombre_archivo=f'./VCF_Mediacoach_Data/{temporada}/{competicion}/{archivo_ids}'
    print(nombre_archivo)
    
    if not os.path.exists(nombre_archivo):
        print(f"Archivo '{nombre_archivo}' no encontrado. Se creará uno nuevo.")
        os.makedirs(os.path.dirname(nombre_archivo), exist_ok=True)
        return []
    
    # CÓDIGO AÑADIDO: Leer el archivo CSV cuando existe
    ids_existentes = []
    try:
        with open(nombre_archivo, 'r', newline='') as file:
            reader = csv.reader(file)
            ids_existentes = [row[0] for row in reader if row]  # Evitar filas vacías
        print(f"Se encontraron {len(ids_existentes)} IDs ya procesados")
        return ids_existentes
    except Exception as e:
        print(f"Error leyendo el archivo {nombre_archivo}: {e}")
        return []


# Función que devuelve una lista con los match_ids que se jugaron antes de los max_match_day y que no estan en la planilla ids_procesados existente
def obtener_ids(max_match_day, season_id, temporada, competition_id, competicion, archivo_ids):
    print(temporada, competicion, archivo_ids)
    ids_existente = leer_ids_csv(temporada, competicion, archivo_ids)
    matches = ejecutar_curl_comando(f"""
    curl --location '{api_url_base}/Championships/seasons/{season_id}/competitions/{competition_id}/matches' \
    {credenciales}
    """)

    if not matches:
        return []
    ids_filtrados = [item['id'] for item in matches if int(item['matchDayNumber']) < int(max_match_day)]
    return [id for id in ids_filtrados if id not in ids_existente]


#Función para descarga un archivos a partir de la url (file_url). Una vez descargado los guarda en la carpeta (carpeta) correspondiente y el nombre (nombre_archivo)
def descargar_y_guardar_archivo(file_url, temporada="Temporada_24_25", competicion="La_Liga", carpeta="Carpeta General", nombre_archivo="Archivo general"):
    file_response = requests.get(file_url)
    ruta_completa = f'./VCF_Mediacoach_Data/{temporada}/{competicion}/{carpeta}/{nombre_archivo}'
    if file_response.status_code == 200:
        #creo el archivo si no existe
        os.makedirs(os.path.dirname(ruta_completa), exist_ok=True)
        with open(ruta_completa, 'wb') as file:
            file.write(file_response.content)
        return True
    return False


# Función que recibe la lista de ids a descargar y descarga los archivos de mediacoach con esa id.
# Cada Id de partido tiene 8 archivos asociados
def descargar_archivos(ids, temporada="Temporada_24_25", competicion="La_Liga",archivo_ids=""):
    nombres_de_archivos = []
    total_archivos = len(ids) * 8  # Suponiendo que cada ID tiene 8 archivos asociados.
    archivos_descargados = 0

    #recorro lista con ids de los partidos
    for i, id in enumerate(ids):
        #Obtengo diccionario con informes de partido
        asset_data = ejecutar_curl_comando(f"curl --location '{api_url_base}/Assets/{id}' {credenciales}")
        if not asset_data:
            continue

        try:
            #Me quedo con las url que me interesan
            file_urls = [asset_data[0]['url'], asset_data[1]['url'], asset_data[5]['url'], asset_data[9]['url'],
                         asset_data[11]['url'], asset_data[12]['url'], asset_data[13]['url'], asset_data[14]['url']]
        except (IndexError, KeyError) as e:
            print(f"Error al procesar el ID {id}: {e}")
            continue  # Continúa con el siguiente ID en caso de error

        #Asigno a cada url una carpeta y una extensión de acuerdo al archivo a descargar
        carpetas = ['Rendimiento', 'Rendimiento', 'postpartidoequipos', 'postpartidojugador',
                    'maximaexigencia', 'maximaexigencia', 'beyondstats', 'maximaexigenciarevisada']
        extensiones = ['xlsx', 'xlsx', 'csv', 'csv', 'xlsx', 'xlsx', 'xml', 'xml']

        #Creo lista con diccionarios: [{file_url, carpeta, extension}] y la recorro
        for j, (file_url, carpeta, extension) in enumerate(zip(file_urls, carpetas, extensiones)):
            #Descargo el archivo y lo guardo en el drive en la carpeta correspondiente
            if file_url and descargar_y_guardar_archivo(file_url, temporada=temporada, competicion=competicion, carpeta=carpeta, nombre_archivo= f'archivo_{id}_descargado_{i}_{j}.{extension}'):
                nombres_de_archivos.append(f'{carpeta}/archivo_{id}_descargado_{i}_{j}.{extension}')
                archivos_descargados += 1
                porcentaje_completado = (archivos_descargados / total_archivos) * 100
                print(f"Progreso: {porcentaje_completado:.2f}% completado ({archivos_descargados}/{total_archivos} archivos)")
        #guardo id del partido en archio csv
        guardar_id_en_csv(id, temporada, competicion, archivo_ids)

    return nombres_de_archivos



# **4- Interfaz de usuario y ejecución del código**

In [12]:
#1 - Preguntar Temporada
temporadas = [{"nombre":"Temporada 24-25", "id":"3a134240-833f-41dd-c6b0-3d6b87479c15", "input":0 },
              {"nombre":"Temporada 23-24", "id":"3a0bf8ee-7f55-aeb6-fd31-273f2d45aefa", "input":1 }]

print("Lista de temporadas dispobibles:")
for temporada in temporadas:
  print(f'{temporada["nombre"]}, seleccione {temporada["input"]}')

input_temporadas = int(input("Selecione una temporada. Pulse q para salir y terminar el proceso"))
while input_temporadas not in [0,1, "q"]:
  print(f"Esa no es una opción válida, selecciona una opción entre 0 y {len(temporadas-1)}")
  input_temporadas = int(input("Selecione una temporada"))

if input_temporadas == 0:
  season_id = temporadas[0]["id"]
  season_name = temporadas[0]["nombre"]
elif input_temporadas ==1:
  season_id = temporadas[1]["id"]
  season_name = temporadas[1]["nombre"]

temporada = season_name.replace(" ", "_").replace("-", "_")
print(f"Ha elegido la temporada {season_name}")


#2 - Preguntar Competición
competiciones = [{"nombre":"La Liga", "id":"39df9ec8-be91-4be5-1925-4b670a4cbed9", "input":0 },
                {"nombre":"La Liga 2", "id":"39df9ec8-becb-86ea-b5e8-600c1b47968d", "input":1 }]

print("Lista de competiciones dispobibles:")
for competicion in competiciones:
  print(f'{competicion["nombre"]}, seleccione {competicion["input"]}')

input_competiciones = int(input("Selecione una competición. Pulse q para salir y terminar el proceso"))
while input_competiciones not in [0,1,"q"]:
  print(f"Esa no es una opción válida, selecciona una opción entre 0 y {len(competiciones-1)}")
  input_competiciones = int(input("Selecione una temporada"))

if input_competiciones == 0:
  competition_id = competiciones[0]["id"]
  competition_name = competiciones[0]["nombre"]
elif  input_competiciones == 1:
  competition_id = competiciones[1]["id"]
  competition_name = competiciones[1]["nombre"]

competicion = competition_name.replace(" ", "_").replace("-", "_")
print(f"Ha elegido la competicion {competition_name}")

archivo_ids = f'ids_procesados_{competicion}.csv'

#3 - Preguntar Max match days
max_match_day = input("Ingrese la cantidad de días de partidos (Ej: 1, 2, 15, etc.) o pulse q para salir y terminar el proceso.")
while ((type(max_match_day)==int) or (max_match_day=="q")):
  print(f"Esa no es una opción válida, ingrese un número entero")
  max_match_day = int(input("Ingrese la cantidad de días de partidos (Ej: 1, 2, 15, etc.):"))
print(f"Ha elegido {max_match_day} días de partidos")


#4 - Ejecutar función con datos proporcionados
ids = obtener_ids(max_match_day, season_id, temporada, competition_id, competicion, archivo_ids)
print(f"Se procesarán {len(ids)} partidos")
if ids:
    nombres_de_archivos = descargar_archivos(ids, temporada=temporada, competicion=competicion, archivo_ids=archivo_ids)

Lista de temporadas dispobibles:
Temporada 24-25, seleccione 0
Temporada 23-24, seleccione 1
Ha elegido la temporada Temporada 24-25
Lista de competiciones dispobibles:
La Liga, seleccione 0
La Liga 2, seleccione 1
Ha elegido la competicion La Liga
Ha elegido 3 días de partidos
Temporada_24_25 La_Liga ids_procesados_La_Liga.csv
./VCF_Mediacoach_Data/Temporada_24_25/La_Liga/ids_procesados_La_Liga.csv
Se encontraron 10 IDs ya procesados
Se procesarán 10 partidos
Progreso: 1.25% completado (1/80 archivos)
Progreso: 2.50% completado (2/80 archivos)
Progreso: 3.75% completado (3/80 archivos)
Progreso: 5.00% completado (4/80 archivos)
Progreso: 6.25% completado (5/80 archivos)
Progreso: 7.50% completado (6/80 archivos)
Progreso: 8.75% completado (7/80 archivos)
Progreso: 10.00% completado (8/80 archivos)
Progreso: 11.25% completado (9/80 archivos)
Progreso: 12.50% completado (10/80 archivos)
Progreso: 13.75% completado (11/80 archivos)
Progreso: 15.00% completado (12/80 archivos)
Progreso: 1